<a href="https://www.kaggle.com/code/dilekdd/rating-product-sorting-reviews-in-amazon?scriptVersionId=196457018" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="text-align: center; font-size: 24px; font-weight: bold; color: green;">
    Rating Product & Sorting Reviews in Amazon
</div>

One of the most significant problems in e-commerce is the accurate calculation of the ratings given to products after purchase. Solving this problem means greater customer satisfaction for the e-commerce site, better product visibility for sellers, and a seamless shopping experience for buyers.


Another issue is the proper ranking of reviews given to products. Misleading reviews being highlighted can directly impact product sales, leading to both financial and customer losses. By solving these two fundamental problems, e-commerce sites and sellers will increase their sales, while customers will complete their purchasing journey without issues.


Data Dictionary


| Variable          | Description                                              |
|-------------------|----------------------------------------------------------|
| reviewerID        | Unique User ID                                            |
| asin              | Product ID                                                |
| reviewerName      | User Name                                                 |
| helpful           | Helpful rating for the review                             |
| reviewText        | Review content                                            |
| overall           | Product rating                                            |
| summary           | Review summary                                            |
| unixReviewTime    | Review timestamp (Unix format)                            |
| reviewTime        | Review date (Raw format)                                  |
| day_diff          | Number of days since the review                           |
| helpful_yes       | Number of helpful votes received by the review            |
| total_vote        | Total number of votes the review received                 |


Observations: 4915
Variables: 12


**Read the dataset and calculate the average rating of the product**

In [1]:
import pandas as pd
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
df = pd.read_csv("/kaggle/input/amazon-review/amazon_review.csv")
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [3]:
df.shape

(4915, 12)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


In [5]:
df["overall"].describe().T

count   4915.00000
mean       4.58759
std        0.99685
min        1.00000
25%        5.00000
50%        5.00000
75%        5.00000
max        5.00000
Name: overall, dtype: float64

In [6]:
df.isnull().sum()


reviewerID        0
asin              0
reviewerName      1
helpful           0
reviewText        1
overall           0
summary           0
unixReviewTime    0
reviewTime        0
day_diff          0
helpful_yes       0
total_vote        0
dtype: int64

In [7]:
df["overall"].mean()

4.587589013224822

**Calculate the Weighted Average Rating by Date**

In [8]:
def time_based_weighted_average(dataframe, w1=28, w2=26, w3=24, w4=22):
    return dataframe.loc[df["day_diff"] <= 30, "overall"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > 30) & (dataframe["day_diff"] <= 90), "overall"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > 90) & (dataframe["day_diff"] <= 180), "overall"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > 180), "overall"].mean() * w4 / 100

time_based_weighted_average(df)

4.6987161061560725

In [9]:
def time_based_weighted_average(dataframe, w1=28, w2=26, w3=24, w4=22):
    return dataframe.loc[dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.25), "overall"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.25)) & (dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.50)), "overall"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.50)) & (dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.75)), "overall"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.75)), "overall"].mean() * w4 / 100

time_based_weighted_average(df)

4.595593165128118

**Create "helpful_no" variable.**
* "total_vote" is the total number of up-down votes given to a review.
* up means helpful.
* There is no helpful_no variable in the dataset, it needs to be generated from the existing variables.

In [10]:
df["helpful_no"] = df["total_vote"] - df["helpful_yes"]
df.head()
df.sort_values(by="helpful_no", ascending=False)[:5]
                            

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no
2909,A10B6G6W3DW1EY,B007WTAJTO,Luopo,"[53, 236]",I know armed with this in my Android tablet an...,4.00000,Win Win situation,1393200000,2014-02-24,287,53,236,183
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,126
2751,A19R7GVV216QKY,B007WTAJTO,Kunchok,"[8, 118]","If price is also double of 64 GB card, then it...",5.00000,Price??,1393286400,2014-02-25,286,8,118,110
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,77
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.00000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,73


**Calculate "score_pos_neg_diff", "score_average_rating" and "wilson_lower_bound" scores and add them to the Dataset**

In [11]:
def score_up_down_diff(up, down):
    return up - down



def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)



# wilson_lower_bound
def wilson_lower_bound(up, down, confidence=0.95):
    """
    Wilson Lower Bound Score calculation

    - The lower limit of the confidence interval to be calculated for the Bernoulli parameter p is accepted as the WLB score.
    - The calculated score is used for product ranking.
    - Note: If the scores are between 1-5, 1-3 is marked as negative, 4-5 as positive and can be made suitable for Bernoulli.This brings some problems with it. For this reason, it is necessary to make a Bayesian average rating.
    
    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

**Creating and calculating 'wlb_score'**

In [12]:
df['wlb_score'] = df.apply(lambda x: wilson_lower_bound(x['helpful_yes'], x['helpful_no']), axis=1)


**Creating and calculating "score_pos_neg_diff"**

In [13]:
df["score_pos_neg_diff"] = df.apply(lambda x: score_up_down_diff(x['helpful_yes'], x['helpful_no']), axis=1)


**Creating and calculating "score_average_rating"**

In [14]:
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x['helpful_yes'], x['helpful_no']), axis=1)


**Identify the first 20 comments and interpret the results.**

In [15]:
df = df[["reviewerID", "overall", "day_diff", "helpful_yes", "helpful_no", "total_vote", "wlb_score", "score_pos_neg_diff", "score_average_rating"]]

df.sort_values(by="wlb_score", ascending=False)[:20]


,reviewerID,overall,day_diff,helpful_yes,helpful_no,total_vote,wlb_score,score_pos_neg_diff,score_average_rating
2031,A12B7ZMXFI6IXY,5.00000,702,1952,68,2020,0.95754,1884,0.96634
3449,AOEAD7DPLZE53,5.00000,803,1428,77,1505,0.93652,1351,0.94884
4212,AVBMZZAFEKO58,1.00000,579,1568,126,1694,0.91214,1442,0.92562
317,A1ZQAQFYSXL5MQ,1.00000,1033,422,73,495,0.81858,349,0.85253
4672,A2DKQQIZ793AV5,5.00000,158,45,4,49,0.80811,41,0.91837
1835,A1J6VSUM80UAF8,5.00000,283,60,8,68,0.78465,52,0.88235
3981,A1K91XXQ6ZEBQR,5.00000,777,112,27,139,0.73214,85,0.80576
3807,AFGRMORWY2QNX,3.00000,649,22,3,25,0.70044,19,0.88000
4306,AOHXKM5URSKAB,5.00000,823,51,14,65,0.67033,37,0.78462
4596,A1WTQUOQ4WG9AI,1.00000,807,82,27,109,0.66359,55,0.75229


In [16]:
df.sort_values(by="score_average_rating", ascending=False)[:20]

,reviewerID,overall,day_diff,helpful_yes,helpful_no,total_vote,wlb_score,score_pos_neg_diff,score_average_rating
4277,A35KXSU6AD1481,5.00000,719,1,0,1,0.20655,1,1.00000
2881,A3VSG5X7GPNNW6,5.00000,1063,1,0,1,0.20655,1,1.00000
1073,A2ZXEKQ2OBZLEE,5.00000,482,1,0,1,0.20655,1,1.00000
445,AIWBDRNBODLEA,4.00000,355,1,0,1,0.20655,1,1.00000
3923,A2PH4RGYVR34L,5.00000,343,1,0,1,0.20655,1,1.00000
435,AUH8I22ITG020,5.00000,867,1,0,1,0.20655,1,1.00000
2901,A28TRYU3FJ039C,5.00000,574,1,0,1,0.20655,1,1.00000
2204,AANX2UN8NPE22,2.00000,542,1,0,1,0.20655,1,1.00000
2206,A3KO3964CNP0XN,5.00000,403,1,0,1,0.20655,1,1.00000
3408,A20WUUD9EDWY4N,5.00000,501,1,0,1,0.20655,1,1.00000


Sorting by score_average_rating does not take into account the cardinality of helpful_yes and helpful_no, not quite helpful.

wlb_score incorporates both helpful_yes and helpful_no into the calculation, highlighting the most positive and negative reviews. Even though some of the reviews are older, wlb_score ranks them in the top 20 because they received a significant number of helpful votes.